* Plot the spatial salinity pattern at peak ebbs with and without Coriolis force.

In [18]:
import numpy as np
import netCDF4 as nc
from salishsea_tools import (nc_tools,viz_tools,tidetools,geo_tools)
import matplotlib.pyplot as plt
import StraTimeSeries as sts
import R_mixStuff as RS
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
sns.set(style="whitegrid")
sns.set_context("notebook", font_scale=2., rc={"lines.linewidth": 2.5})
%matplotlib inline

In [2]:
grid6 = nc.Dataset('/ocean/jieliu/research/meopar/river-treatment/bathy_meter_SalishSea6.nc')
X = grid6.variables['nav_lon'][:, :]
Y = grid6.variables['nav_lat'][:, :]
bathy = grid6.variables['Bathymetry'][:, :]
## tmask
mesh = nc.Dataset('/data/jieliu/MEOPAR/river-treatment/oct8_101e061e05/mesh_mask.nc')
tmask = mesh.variables['tmask'][0,:,380:510,240:397]
np_mask = np.abs(1-tmask) 
e3t = mesh.variables['e3t'][0,:,380:510,240:397]

In [3]:
## load grid_T model results
octall_T = nc.Dataset('/data/jieliu/MEOPAR/SurfaceCurrent/Oct2014combineall/oct2014all_T.nc')
mayall_T = nc.Dataset('/data/jieliu/MEOPAR/SurfaceCurrent/May2015combineall/may2015all_T.nc')
janall_T = nc.Dataset('/data/jieliu/MEOPAR/SurfaceCurrent/Jan2016combineall/jan2016all_T.nc')

octnof_T = nc.Dataset('/data/jieliu/MEOPAR/SurfaceCurrent/Oct2014nof/oct2014nof_T.nc')
maynof_T = nc.Dataset('/data/jieliu/MEOPAR/SurfaceCurrent/May2015nof/may2015nof_T.nc')
jannof_T = nc.Dataset('/data/jieliu/MEOPAR/SurfaceCurrent/Jan2016nof/jan2016nof_T.nc')

In [7]:
def determine_plume_method2_onlysalinity(grid_T,tst,ted,dep_ind):
    """Similar to method1, but this time the S_ref is the spatial mean 
    of the 2.5*2.5 km in the northern of my subdomain and not plot them, only get the
    threshold salinity"""
    S = grid_T.variables['vosaline']
    lon = grid_T.variables['nav_lon']
    lat = grid_T.variables['nav_lat']
    S_aveN = np.nanmean(S[tst:ted,:,-5:,0:5],axis = 0) ## first temporal average of northern region
    S_ref = np.nanmean(S_aveN[dep_ind]) ## then spatial average
    S_thre = 1.14*S_ref-4.8
    S_aveW = np.nanmean(S[tst:ted,dep_ind],axis = 0) ## temporal averaged of the whole domain
    S_aveW = np.ma.masked_values(S_aveW, 0) ## mask 0 value
    return S_thre, S_aveW

In [6]:
def Salinity(grid_T1,grid_T2,grid_U1,grid_U2,grid_V1,grid_V2,t1,t2,colormap,st,dep,cmin = -6,cmax = 6,ave = False):
    """Quiver velocity on top of surface salinity"""
    fig,axs = plt.subplots(1,3,figsize=(20,6))
    theta = np.pi*29/180
    S = {};U = {};V= {}
    for gridT, gridU,gridV,run, ax in zip([grid_T1,grid_T2],[grid_U1,grid_U2],[grid_V1,grid_V2],['all','nof'],[axs[0],axs[1]]):
        lon = gridT.variables['nav_lon']
        lat = gridT.variables['nav_lat']
        if ave ==False:
            S[run] =gridT.variables['vosaline'][t1,0]
            S[run] = np.ma.masked_values(S[run],0)
        else:
            S[run] =np.ma.masked_values(np.nanmean(gridT.variables['vosaline'][t1:t2+1,0],axis = 0),0)
            
        mesh=ax.pcolormesh(lon,lat,S[run],vmin =0,vmax = 33,cmap=colormap) 
        cbar = fig.colorbar(mesh,ax = ax)
        cbar.set_label('Practical Salinity')
        if ave == False:
            S_thre,W = determine_plume_method2_onlysalinity(gridT,t1,t1+1,dep)
        else:
            S_thre,W = determine_plume_method2_onlysalinity(gridT,t1,t2+1,dep)
        CS3 = ax.contour(lon[:],lat[:],W, levels = [S_thre], colors='k',linewidths=(3,))
        plt.clabel(CS3,fmt='%2.1f', colors='k',fontsize = 14)
        viz_tools.plot_land_mask(ax,grid6,coords='map',color='burlywood')
        viz_tools.plot_coastline(ax, grid6, coords = 'map')
        ax.set_xlim([-124.,-122.8])
        ax.set_xticks([-124,-123.6,-123.2,-122.8])
        ax.set_xticklabels([-124,-123.6,-123.2,-122.8])
        ax.set_ylim([48.9,49.5])
        ## add minimum salinity plot if available
    sal_diff = S['all'] - S['nof']
    ax = axs[2]
    mesh=ax.pcolormesh(lon,lat,sal_diff,cmap='RdBu_r',vmin=cmin, vmax=cmax)
    cbar = fig.colorbar(mesh,ax = ax)
    cbar.set_label('Difference of Practical Salinity')
    viz_tools.plot_land_mask(ax,grid6,coords='map',color='burlywood')
    viz_tools.plot_coastline(ax, grid6, coords = 'map')
    ax.set_xlim([-124.,-122.8])
    ax.set_xticks([-124,-123.6,-123.2,-122.8])
    ax.set_xticklabels([-124,-123.6,-123.2,-122.8])
    ax.set_ylim([48.9,49.5])

In [8]:
U = {}
for month,gridU in zip(['jan','oct','may'],[janall_U,octall_U,mayall_U]):
    U[month] = gridU.variables['vozocrtx']
SSH = {}
for month,gridT in zip(['jan','oct','may'],[janall_T,octall_T,mayall_T]):
    SSH[month] = gridT.variables['sossheig']

In [22]:
theta = np.pi*29/180
grid_T_alls = [janall_T,octall_T,mayall_T]
grid_T_nofs = [jannof_T,octnof_T,maynof_T]
colormap = 'plasma_r'
dep = 0
cmin = -6
cmax = 6

In [ ]:
fig,axs = plt.subplots(3,3,figsize=(20,18))
    
for i, t1 in zip(np.arange(3), [741,679,685]):
    S = {};
    if i == 0:
        letters = ['a. Jan With Coriolis force', 'd. Jan Without Coriolis force', 'g. Jan With - Without Coriolis force']
    elif i ==1:
        letters = ['b. Oct With Coriolis force', 'e. Oct Without Coriolis force', 'h. Oct With - Without Coriolis force']
    elif i ==2:
        letters = ['c. May With Coriolis force', 'f. May Without Coriolis force', 'i. May With - Without Coriolis force']
    for gridT,run, ax,letter in zip([grid_T_alls[i],grid_T_nofs[i]],['all','nof'],[axs[i,0],axs[i,1]],letters):
        lon = gridT.variables['nav_lon']
        lat = gridT.variables['nav_lat']
        S[run] =gridT.variables['vosaline'][t1,0]
        S[run] = np.ma.masked_values(S[run],0) 
        mesh=ax.pcolormesh(lon,lat,S[run],vmin =0,vmax = 33,cmap=colormap) 
        ax.text(0,1.03,letter, transform = ax.transAxes,fontsize = 18)
        cbar = fig.colorbar(mesh,ax = ax)
        cbar.set_label('Practical Salinity')
        S_thre,W = determine_plume_method2_onlysalinity(gridT,t1,t1+1,dep)
        CS3 = ax.contour(lon[:],lat[:],W, levels = [S_thre], colors='k',linewidths=(3,))
        plt.clabel(CS3,fmt='%2.1f', colors='k',fontsize = 14)
        if letter == 'c. May With Coriolis force':
            ax.set_xlabel('Longitude')
            ax.set_ylabel('Latitude')
    sal_diff = S['all'] - S['nof']
    ax = axs[i,2]
    mesh=ax.pcolormesh(lon,lat,sal_diff,cmap='RdBu_r',vmin=cmin, vmax=cmax)
    ax.text(0,1.03,letters[-1], transform = ax.transAxes,fontsize = 18)
    cbar = fig.colorbar(mesh,ax = ax)
    cbar.set_label('Difference of Practical Salinity')
    for ax in [axs[i,0],axs[i,1],axs[i,2]]:
        viz_tools.plot_land_mask(ax,grid6,coords='map',color='burlywood')
        viz_tools.plot_coastline(ax, grid6, coords = 'map')
        ax.set_xlim([-124.,-122.8])
        ax.set_xticks([-124,-123.6,-123.2,-122.8])
        ax.set_xticklabels([-124,-123.6,-123.2,-122.8])
        ax.set_ylim([48.9,49.5])